In [1]:
import statsmodels.api as sm 
from statsmodels.tsa.stattools import grangercausalitytests
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from statsmodels.graphics import tsaplots
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.stats.stattools import durbin_watson
from sklearn.metrics import mean_absolute_error

def excel_to_pandas(path):
    """This function converts a csv file into a pandas dataframe. 
    
    Parameters:
        Path: Where the csv file is located on the device
    
    Returns a pandas dataframe.
    """
    New = pd.read_excel(path)
    return New 

//anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


**1. Calling in the combined Yahoo Finance and Google Trends data**

In [2]:
Trends_and_Price = excel_to_pandas('/Users/jacobkuhlman/Desktop/Final Project DA /Google Trends and Price/Trends_and_Price_Year.xlsx')

**2. Extracting just the Dogecoin data**

In [6]:
Dogecoin_Trends = Trends_and_Price.loc[Trends_and_Price['Cryptocurrency_type'] == 'Dodgecoin']
Dogecoin_Trends = Dogecoin_Trends[['Date','Close', 'Search Volume', 'Volume']]
Dogecoin_Trends = Dogecoin_Trends.set_index('Date')
Dogecoin_Trends = Dogecoin_Trends.fillna(method = 'pad')

**3. Creating a training and testing set**

In [8]:
n_obs=15
#n_obs=50
X_train, X_test = Dogecoin_Trends[0:-n_obs], Dogecoin_Trends[-n_obs:]
print(X_train.shape, X_test.shape)

(1735, 3) (15, 3)


**4. Conducting the ADF test**

In [9]:
X_train_transformed = X_train.diff().dropna()
transform_data = X_train.diff().dropna()

def augmented_dickey_fuller_statistics(time_series):
    result = adfuller(time_series.values)
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

        
print('Augmented Dickey-Fuller Test: Close Price Time Series')
#augmented_dickey_fuller_statistics(X_train_transformed['Close'])

print('Augmented Dickey-Fuller Test: Search Volume Time Series')
#augmented_dickey_fuller_statistics(X_train_transformed['Search Volume'])

Augmented Dickey-Fuller Test: Close Price Time Series
Augmented Dickey-Fuller Test: Search Volume Time Series


**5. Conducting the Granger Causality test**

In [11]:
maxlag = 12
test = 'ssr-chi2test'
def grangers_causality_matrix(X_train, variables, test = 'ssr_chi2test', verbose=False):
    dataset = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables) 
    for c in dataset.columns:
        for r in dataset.index:
            test_result = grangercausalitytests(X_train[[r,c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            dataset.loc[r,c] = min_p_value
            #dataset.columns = [var + '_x' for var in variables]
            #dataset.index = [var + '_y' for var in variables]
    return dataset
grangers_causality_matrix(Dogecoin_Trends, variables = Dogecoin_Trends.columns)

,Close,Search Volume,Volume
Close,1.0000,0.1272,0.0000
Search Volume,0.0148,1.0000,0.3466
Volume,0.0000,0.0470,1.0000
